<a href="https://colab.research.google.com/github/Chaitanya203163/Root_Optimization/blob/Step5/Step_5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
from geopy.distance import geodesic
import folium
import heapq

def calculate_distance(lat1, lon1, lat2, lon2):
    coords_1 = (lat1, lon1)
    coords_2 = (lat2, lon2)
    return geodesic(coords_1, coords_2).km

def allocate_construction_sites(construction_df, retailer_df):
    allocation = {retailer_id: [] for retailer_id in retailer_df['Retailer_id']}


    for _, construction_row in construction_df.iterrows():
        min_distance = float('inf')
        closest_retailer = None

        for _, retailer_row in retailer_df.iterrows():
            distance = calculate_distance(construction_row['Latitude'], construction_row['Longitude'],
                                          retailer_row['Latitude'], retailer_row['Longitude'])
            if distance < min_distance:
                min_distance = distance
                closest_retailer = retailer_row['Retailer_id']

        allocation[closest_retailer].append(construction_row['Construction_id'])

    return allocation

def main():

    construction_df = pd.read_excel('/content/construction_sites.xlsx')
    retailer_df = pd.read_excel('/content/retailers_data.xlsx')


    allocation = allocate_construction_sites(construction_df, retailer_df)


    retailer_ids = list(retailer_df['Retailer_id'])
    max_allocated_sites = max(len(allocated_sites) for _, allocated_sites in allocation.items())
    allocation_dict = {retailer_id: allocated_sites + [''] * (max_allocated_sites - len(allocated_sites))
                       for retailer_id, allocated_sites in allocation.items()}
    allocation_df = pd.DataFrame(allocation_dict)
    allocation_df.to_excel('construction_allocation.xlsx', index=False)


    m = folium.Map(location=[retailer_df['Latitude'].mean(), retailer_df['Longitude'].mean()], zoom_start=13)


    for _, row in retailer_df.iterrows():
        folium.Marker([row['Latitude'], row['Longitude']], popup=row['Retailer_id']).add_to(m)
        folium.Circle([row['Latitude'], row['Longitude']], radius=1000, color='blue', fill=True, fill_color='blue').add_to(m)

    for _, row in construction_df.iterrows():
        folium.Circle([row['Latitude'], row['Longitude']], radius=50, color='red', fill=True, fill_color='red').add_to(m)

    m.save('map.html')

if __name__ == "__main__":
    main()
